In [5]:
# prompt: load the SMV_VAS Bauchi file
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import os
from IPython.display import display # Import display if not already present

# Load the Excel file
df = pd.read_excel('Dataset/SMC_VAS BAUCHI BASELINE SURVEY 2025_Play.xlsx')

# Prepare the needed data
df['selected_chi_monthe'] = pd.to_numeric(df['selected_chi_monthe'], errors='coerce')
df['status_treatment_VASnow'] = df['status_treatment_VASnow'].str.strip().str.upper()

# Categorize age into 6-11 and 12-59 months
df['age_group'] = pd.cut(df['selected_chi_monthe'],
                         bins=[5, 11, 59],
                         labels=["6-11 months", "12-59 months"])

# Filter valid rows
filtered_df = df[df['status_treatment_VASnow'].isin(['YES', 'NO']) & df['age_group'].notna()]

# Cross-tabulation
crosstab = pd.crosstab(filtered_df['age_group'], filtered_df['status_treatment_VASnow'])

# Calculate percentages
row_totals = crosstab.sum(axis=1)
percent_yes = round(crosstab['YES'] / row_totals * 100, 1)
percent_no = round(crosstab['NO'] / row_totals * 100, 1)

# Format counts and percentages
output_df = pd.DataFrame({
    "Characteristics": crosstab.index,
    "No (%)": [f"{crosstab.loc[idx, 'NO']} ({percent_no[idx]}%)" for idx in crosstab.index],
    "Yes (%)": [f"{crosstab.loc[idx, 'YES']} ({percent_yes[idx]}%)" for idx in crosstab.index]
})

# Chi-square test
chi2, p, _, _ = chi2_contingency(crosstab)
output_df["P-value"] = ""
output_df.loc[0, "P-value"] = round(p, 4)

# Add section title row
output_df = pd.concat([
    pd.DataFrame([{"Characteristics": "Childs age_months", "No (%)": "", "Yes (%)": "", "P-value": ""}]),
    output_df
], ignore_index=True)

# Replace the ace_tools line with display(output_df)
display(output_df)

,Characteristics,No (%),Yes (%),P-value
0,Childs age_months,,,
1,6-11 months,67 (31.9%),143 (68.1%),0.0
2,12-59 months,362 (13.9%),2243 (86.1%),
